# Data Collection

In [104]:
import pandas as pd

df = pd.read_csv("../data/yahoo_answers.csv")

human_answers = df["answer"].tolist()
for i, answer in enumerate(human_answers):
  with open("../data/responses.csv", "a") as f:
    f.write(f'{i},"{answer}",human\n')


In [120]:
df = pd.read_csv("../data/yahoo_answers.csv")

df["question"] = df["header"] + " " + df["subheader"] if type(df["subheader"]) == "string" else ""
questions = df["question"].tolist()

DELIMETER = "|"
for i, answer in enumerate(df["answer"]):
  if type(answer) == str:
    answer = answer.replace("|", "")
  with open("../data/responses.csv", "a") as f:
    f.write(f'{i}{DELIMETER}"{answer}"{DELIMETER}human\n')

In [ ]:
from time import sleep
import os
import pandas as pd
import openai
from dotenv import load_dotenv

load_dotenv

openai.api_key = os.getenv("OPENAI_API_KEY")

df = pd.read_csv("../data/yahoo_answers.csv")
df["question"] = df["header"] + (" " + df["subheader"] if type(df["subheader"]) == str else "")
questions = df["question"]

DELIMETER = "|"

responses_df = pd.read_csv("../data/responses.csv", sep=DELIMETER)
num_answers = len(responses_df)
responses_df = responses_df[responses_df["category"] == "ChatGPT"]

for i, question in enumerate(questions):
  if i % 100 == 0:
    print(f"Processing question {i} of {len(questions)}")
  if i < len(responses_df):
    continue
  should_break = False
  while not should_break:
    try:
      response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
              {"role": "system", "content": "Respond to the following question like you are a human answering a short essay prompt."},
              {"role": "user", "content": question},
          ]
      )
      should_break = True
    except Exception as e:
      # if the OpenAI API is overloaded, wait 2 minutes and try again
      print(e)
      sleep(120)

  answer = response["choices"][0]["message"]["content"]
  items_to_remove = [
    "As an AI language model, ",
    "Sorry, as an AI language model ",
    "I'm sorry, as an AI language model, ",
    "I'm sorry, but as an AI language model, ",
    "I do not have personal preferences or emotions. ",
    "I do not have personal preferences or emotions, but ",
    "I do not have personal beliefs or opinions. ",
    "As a language model AI, ",
    "|",
    "\n"
  ]
  for item in items_to_remove:
    answer = answer.replace(item, "")
  answer = answer.replace('"', "'")
  answer = answer.replace(":", ": ")

  with open("../data/responses.csv", "a") as f:
    f.write(f'{num_answers+i}{DELIMETER}"{answer}"{DELIMETER}ChatGPT\n')

In [3]:
import math
import random
import pandas as pd

df = pd.read_csv("../data/responses.csv", sep="|")
ai = df[df["category"] == "ChatGPT"]
human = df[df["category"] == "human"]
number_to_use = math.floor((len(ai) / len(human)) * len(human))

human_to_use = random.sample(human["text"].tolist(), number_to_use)
ai_to_use = ai["text"].tolist()

# # create a dataframe with human_to_use with the category "human" and ai_to_use with the category "ChatGPT"
data = []
data += [[item, "human"] for item in human_to_use]
data += [[item, "ChatGPT"] for item in ai_to_use]
df = pd.DataFrame(data, columns=["text", "category"])

df.to_csv("../data/data.csv", sep="|")

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy

nlp = spacy.load("en_core_web_sm")

df = pd.read_csv("../data/data.csv", sep="|")

def tokenize_text(text):
    doc = nlp(str(text))
    return ",".join([token.text for token in doc])

df["tokens"] = df["text"].apply(tokenize_text)


In [ ]:
# get the median number of tokens in each category
human_median = df[df["category"] == "human"]["tokens"].apply(lambda x: len(x)).median()
ai_median = df[df["category"] == "ChatGPT"]["tokens"].apply(lambda x: len(x)).median()

# graph the median length of each category
plt.bar(["human", "ChatGPT"], [human_median, ai_median])